In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [76]:
def EfficientTesting (N,f=0.01,K=None):
    """ Run efficient testing for population size N,
        with fraction f positive and in per-subject K test repeats.
        If K is None, set to optimal."""
    true_positive = (np.random.uniform(0,1,N)<f)
    optef=-f*np.log(f)/np.log(2)-(1-f)*np.log(1-f)
    print ("Formally optimal efficiency:",optef)
    lat=[]
    for i,method in enumerate([Method1, Method2]):
        print ('-------------------------- method ',i)
        ## now collect all results
        results, Ng, latex = method(true_positive, f, K) 
        lat.append(latex)
        tp = results[true_positive].sum()
        tn = (~results[~true_positive]).sum()
        fp = results[~true_positive].sum()
        fn = (~results[true_positive]).sum()
        print ("True positives: %i True negatives: %i False Positives:%i, False Negatives: %i"%(tp,tn,fp,fn))
        if (i==0):
            print ("Total tests required:",Ng)
            print ("Efficiency total: %f"%(Ng/N))
        elif (i==1):
            Tall=Ng+tp+fp
            print ("Total tests required for perfect result %i+%i = %i"%(Ng,tp+fp,Tall))
            print ("Efficiencies phase 1: %f, total: %f"%(Ng/N, Tall/N))
    print ("Latex: %i & %s & %s & %3.3f \\\\ "%(true_positive.sum(), lat[0],lat[1],optef))
    
def Method1(true_positive,f,K):
    N=len(true_positive)
    suspect=list(range(N))
    it=0
    Ninf = f*N
    tcount = 0
    lx=""
    while True:
        C = len(suspect)
        fc = Ninf / C
        M = np.int(-np.log(2)/np.log(1-fc))
        lx+= str(M)+", "
        if (M==0): M=1
        newsuspect = []
        cres = []
        for c in range(C//M+1):
            if c<C//M:
                cut = suspect[c*M:(c+1)*M]
            else:
                cut = suspect[c*M:]
            res=np.any(true_positive[cut])
            tcount += 1
            cres.append(res)
            if res:
                newsuspect += cut
        print ("Iteration %i :: C = %i M = %i  mean rate: %f  "%(it,C,M,np.array(cres).mean()))
        it += 1                                    
        suspect = newsuspect
        if M == 1: 
            break
    results=np.zeros(N,np.bool)
    results[suspect] = True
    return results, tcount, "%i & %s & %i & %3.3f "%(it,lx[:-2],tcount, tcount/N)
                                
                                    
def Method2(true_positive, f, K):
    N = len(true_positive)
    M = np.int(-np.log(2)/np.log(1-f)+1)
    if K is None:
        K = np.int (-np.log(-np.log(1-f)/(1-f)/np.log(2)**2)/np.log(2) +0.5)
    
    Ng = (N//M+1)*K 
    print ("N=%i M=%i K=%i Ng=%i"%(N,M,K,Ng))
    table = np.zeros((Ng,N),int)
    print ("Making groups...")
    gleft = np.ones(Ng,int)*M ## slots left in each group
    for s in range(N):
        if (s%10000==0):
            print ('...',s,table[:10,:].sum(axis=1))
        while True:
            ## let's propose where we want to put this person
            proposal = np.random.choice(Ng, K, p=gleft/gleft.sum(),replace=False)
            ## check if some other subject already came up with exactly the samge grouping?
            if not (np.any(table[proposal,:].sum(axis=0) == K)): ## if any other s has exactly the same groups...
                break
        table[proposal,s] = 1
        gleft[proposal] -= 1
    print ("Collecting results...")
    group_results = np.array([np.any(true_positive[np.where(row==1)]) for row in table])
    print ("Mean rate of positive result in groups:",group_results.mean())
    # now collect what results for subjects are
    results=np.array([np.all(group_results[np.where(column==1)]) for column in table.T])

    return results,Ng, "%i & %i & %i & %i/%i & %3.3f/%3.3f "%(M,K,results.sum()-true_positive.sum(),Ng,Ng+results.sum(), Ng/N,(Ng+results.sum())/N)
 

In [74]:
EfficientTesting(100000,f=1e-2)

Formally optimal efficiency: 0.07638839439271368
-------------------------- method  0
Iteration 0 :: C = 100000 M = 68  mean rate: 0.523453  
Iteration 1 :: C = 52360 M = 35  mean rate: 0.583166  
Iteration 2 :: C = 30555 M = 20  mean rate: 0.602094  
Iteration 3 :: C = 18395 M = 12  mean rate: 0.615786  
Iteration 4 :: C = 11327 M = 7  mean rate: 0.597282  
Iteration 5 :: C = 6769 M = 4  mean rate: 0.591849  
Iteration 6 :: C = 4008 M = 2  mean rate: 0.527182  
Iteration 7 :: C = 2114 M = 1  mean rate: 0.509220  
True positives: 1077 True negatives: 98923 False Positives:0, False Negatives: 0
Total tests required: 13461
Efficiency total: 0.134610
-------------------------- method  1
N=100000 M=69 K=6 Ng=8700
Making groups...
... 0 [0 0 0 0 0 0 0 0 0 0]
... 10000 [10  2 10  7  7  7  6 11  6  6]
... 20000 [19 10 12 20 14 15 12 16 11 12]
... 30000 [28 14 19 25 19 18 18 23 19 18]
... 40000 [35 25 27 34 23 29 24 28 29 27]
... 50000 [38 29 33 37 34 37 28 34 35 31]
... 60000 [45 36 40 42 40 

In [77]:
EfficientTesting(100000,f=1e-3)

Formally optimal efficiency: 0.010965284117912038
-------------------------- method  0
Iteration 0 :: C = 100000 M = 692  mean rate: 0.517241  
Iteration 1 :: C = 51560 M = 357  mean rate: 0.551724  
Iteration 2 :: C = 28355 M = 196  mean rate: 0.641379  
Iteration 3 :: C = 18163 M = 125  mean rate: 0.616438  
Iteration 4 :: C = 11250 M = 77  mean rate: 0.625850  
Iteration 5 :: C = 7084 M = 48  mean rate: 0.621622  
Iteration 6 :: C = 4396 M = 30  mean rate: 0.653061  
Iteration 7 :: C = 2880 M = 19  mean rate: 0.625000  
Iteration 8 :: C = 1797 M = 12  mean rate: 0.626667  
Iteration 9 :: C = 1125 M = 7  mean rate: 0.559006  
Iteration 10 :: C = 628 M = 3  mean rate: 0.485714  
Iteration 11 :: C = 306 M = 1  mean rate: 0.345277  
True positives: 106 True negatives: 99894 False Positives:0, False Negatives: 0
Total tests required: 2003
Efficiency total: 0.020030
-------------------------- method  1
N=100000 M=693 K=9 Ng=1305
Making groups...
... 0 [0 0 0 0 0 0 0 0 0 0]
... 10000 [74 6